In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

def load_csv():
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        global df, filtered_df
        df = pd.read_csv(file_path, header=None, names=["words"])
        df = df.dropna()
        df = df[~df["words"].str.isupper()]
        df = df[df["words"].str.len() >= 4]
        df = df[df["words"].str.match("^[a-zA-Z]+$")]
        messagebox.showinfo("Success", "CSV file loaded successfully!")

def start_game():
    global filtered_df
    try:
        letter_count = int(letter_entry.get())
        chances = int(chances_entry.get())
        if letter_count < 4 or chances < 1:
            raise ValueError
        
        filtered_df = df[df["words"].str.len() == letter_count].copy()
        filtered_df.reset_index(drop=True, inplace=True)
        remaining_chances.set(chances)
        output_box.delete("1.0", tk.END)
        output_box.insert(tk.END, f"Game started! {len(filtered_df)} words loaded.\n")
    except ValueError:
        messagebox.showerror("Error", "Invalid input! Enter valid numbers for letters and chances.")

def get_filtered_words(input_word, condition):
    global filtered_df
    letter_count = len(input_word)
    for i in range(letter_count):
        if condition[i].lower() == "g":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: x.lower()[i] == input_word[i].lower())]
        elif condition[i].lower() == "y":
            filtered_df = filtered_df[
                filtered_df['words'].apply(lambda x: input_word[i].lower() in x.lower() and x.lower()[i] != input_word[i].lower())
            ]
        elif condition[i].lower() == "x":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: input_word[i].lower() not in x.lower())]
    return filtered_df

def apply_filter():
    global filtered_df
    if remaining_chances.get() <= 0:
        messagebox.showinfo("Game Over", "No more chances left!")
        return
    
    input_word = word_entry.get().strip()
    condition = condition_entry.get().strip()
    letter_count = int(letter_entry.get())
    
    if len(input_word) != letter_count or len(condition) != letter_count:
        messagebox.showerror("Error", "Word and condition must match the letter count!")
        return
    if not set(condition.lower()).issubset({"g", "x", "y"}):
        messagebox.showerror("Error", "Condition can only contain G, X, or Y!")
        return
    
    filtered_df = get_filtered_words(input_word.lower(), condition)
    remaining_chances.set(remaining_chances.get() - 1)
    
    output_box.delete("1.0", tk.END)
    output_box.insert(tk.END, f"Remaining words: {len(filtered_df)}\n")
    output_box.insert(tk.END, "\n".join(filtered_df['words'].sample(min(len(filtered_df), 15)).tolist()))

# GUI Setup
root = tk.Tk()
root.title("Word Filtering Game")

tk.Button(root, text="Load CSV", command=load_csv).pack(pady=5)

tk.Label(root, text="Letters:").pack()
letter_entry = tk.Entry(root)
letter_entry.pack()

tk.Label(root, text="Chances:").pack()
chances_entry = tk.Entry(root)
chances_entry.pack()

tk.Button(root, text="Start Game", command=start_game).pack(pady=5)

remaining_chances = tk.IntVar()

word_frame = tk.Frame(root)
tk.Label(word_frame, text="Word:").pack(side=tk.LEFT)
word_entry = tk.Entry(word_frame)
word_entry.pack(side=tk.LEFT)
word_frame.pack()

tk.Label(root, text="Condition (G, X, Y):").pack()
condition_entry = tk.Entry(root)
condition_entry.pack()

tk.Button(root, text="Apply Filter", command=apply_filter).pack(pady=5)

output_box = scrolledtext.ScrolledText(root, width=50, height=10)
output_box.pack()

root.mainloop()


In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext

def load_csv():
    file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    if file_path:
        global df, filtered_df
        df = pd.read_csv(file_path, header=None, names=["words"])
        df = df.dropna()
        df = df[~df["words"].str.isupper()]
        df = df[df["words"].str.len() >= 4]
        df = df[df["words"].str.match("^[a-zA-Z]+$")]
        messagebox.showinfo("Success", "CSV file loaded successfully!")

def start_game():
    global filtered_df, entry_boxes
    try:
        letter_count = int(letter_entry.get())
        chances = int(chances_entry.get())
        if letter_count < 4 or chances < 1:
            raise ValueError
        
        filtered_df = df[df["words"].str.len() == letter_count].copy()
        filtered_df.reset_index(drop=True, inplace=True)
        remaining_chances.set(chances)
        
        for widget in input_frame.winfo_children():
            widget.destroy()
        
        entry_boxes = []
        for r in range(chances):
            row_boxes = []
            for c in range(letter_count):
                box = tk.Entry(input_frame, width=6, font=("Arial", 20, "bold"), justify='center')
                box.grid(row=r, column=c, padx=6, pady=6)
                box.bind("<KeyRelease>", lambda event, row=r, col=c: move_cursor(event, row, col))
                row_boxes.append(box)
            entry_boxes.append(row_boxes)
        
        output_box.delete("1.0", tk.END)
        output_box.insert(tk.END, f"Game started! {len(filtered_df)} words loaded.\n")
        root.geometry("800x600")
    except ValueError:
        messagebox.showerror("Error", "Invalid input! Enter valid numbers for letters and chances.")

def move_cursor(event, row, col):
    if event.keysym == "BackSpace":
        if entry_boxes[row][col].get() == "" and col > 0:
            entry_boxes[row][col - 1].focus_set()
    elif event.keysym.isalnum() and col < len(entry_boxes[row]) - 1:
        entry_boxes[row][col + 1].focus_set()

def get_filtered_words(input_word, condition):
    global filtered_df
    letter_count = len(input_word)
    for i in range(letter_count):
        if condition[i].lower() == "g":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: x.lower()[i] == input_word[i].lower())]
        elif condition[i].lower() == "y":
            filtered_df = filtered_df[
                filtered_df['words'].apply(lambda x: input_word[i].lower() in x.lower() and x.lower()[i] != input_word[i].lower())
            ]
        elif condition[i].lower() == "x":
            filtered_df = filtered_df[filtered_df['words'].apply(lambda x: input_word[i].lower() not in x.lower())]
    return filtered_df

def apply_filter():
    global filtered_df
    if remaining_chances.get() <= 0:
        messagebox.showinfo("Game Over", "No more chances left!")
        return
    
    row_index = -remaining_chances.get()
    input_word = "".join([box.get().strip() for box in entry_boxes[row_index]])
    condition = condition_entry.get().strip()
    letter_count = int(letter_entry.get())
    
    if len(input_word) != letter_count or len(condition) != letter_count:
        messagebox.showerror("Error", "Word and condition must match the letter count!")
        return
    if not set(condition.lower()).issubset({"g", "x", "y"}):
        messagebox.showerror("Error", "Condition can only contain G, X, or Y!")
        return
    
    filtered_df = get_filtered_words(input_word.lower(), condition)
    remaining_chances.set(remaining_chances.get() - 1)
    
    color_boxes(row_index, condition)
    
    output_box.delete("1.0", tk.END)
    output_box.insert(tk.END, f"Remaining words: {len(filtered_df)}\n")
    output_box.insert(tk.END, "\n".join(filtered_df['words'].tolist()))

def color_boxes(row, condition):
    colors = {"g": "green", "y": "gold", "x": "gray"}
    for i, box in enumerate(entry_boxes[row]):
        box.config(bg=colors.get(condition[i].lower(), "white"), fg="black" if condition[i].lower() == "y" else "white")

# GUI Setup
root = tk.Tk()
root.title("Word Filtering Game")
root.geometry("800x600")
root.eval('tk::PlaceWindow . center')

tk.Button(root, text="Load CSV", font=("Arial", 16), command=load_csv).pack(pady=5)

tk.Label(root, text="Letters:", font=("Arial", 16)).pack()
letter_entry = tk.Entry(root, font=("Arial", 16))
letter_entry.pack()

tk.Label(root, text="Chances:", font=("Arial", 16)).pack()
chances_entry = tk.Entry(root, font=("Arial", 16))
chances_entry.pack()

tk.Button(root, text="Start Game", font=("Arial", 16), command=start_game).pack(pady=5)

remaining_chances = tk.IntVar()
input_frame = tk.Frame(root)
input_frame.pack()

tk.Label(root, text="Condition (G, X, Y):", font=("Arial", 16)).pack()
condition_entry = tk.Entry(root, font=("Arial", 16))
condition_entry.pack()

tk.Button(root, text="Apply Filter", font=("Arial", 16), command=apply_filter).pack(pady=5)

output_box = scrolledtext.ScrolledText(root, width=60, height=15, font=("Arial", 16))
output_box.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'words'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/tkinter/__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/s9/0x0wmn8j1rg6kj1_92zsq8pc0000gn/T/ipykernel_10223/3160098255.py", line 86, in apply_f